# Root finding

In [5]:
import numpy as np
import matplotlib.pyplot as plt

## Método de bisección

Se basa en el **teorema del valor medio**: Si $f(x)$ es una función continua en el intervalo $[a,b]$ y $k$ es un número entre $f(a)$ y $f(b)$, entonces existe un número $c$ tal que $f(c) = k$

Escogemos un intervalo que contenga la raiz, para esto debemos escoger $[a,b]$ de forma que $f(a)f(b) < 0$, pues, esto garantiza que uno de los terminos es negativo y por ende, hay un cambio de signo entre $a$ y $b$.

Se revisa si $a$ y $b$ son raíces, si no, se elige un punto medio entre estos valores de abcisa. $c =\frac{a+b}{2}$

Revisamos si la raiz está entre $[a,c]$ o si está entre $c,b$. Elegido el nuevo intervalo, repetimos el procedimiento hasta que $|f(c)| < \epsilon$

In [6]:
def biseccion(f,a,b,eps = 1e-8,Nmax = 100):
    '''Usa el algoritmo de bisección para encontrar las raices de una función dada'''
    
    #Verificamos que el intervalo [a,b] cumpla las condiciones
    if a > b or f(a)*f(b) >= 0:
        print('Escoger un mejor intervalo!')
        r = None
    
    #Si cumple las condiciones, verificamos que la raiz no esté en los extremos
    elif abs(f(a)) <= eps: 
        r = a
    elif abs(f(b)) <= eps: 
        r = b
            
    #Si los extremos no son raiz, entonces implementamos el algoritmo
    else:
        r = (a+b)/2 #escogemos punto medio
        i = 0
        while abs(f(r)) > eps and i < Nmax:
            i += 1
        
            #verificamos en qué intervalo está la raiz

            if f(a)*f(r) < 0:
            #La raiz está en el intervalo [a,r]
                b = r
                r = (a+b)/2

            #La raiz está en el intervalo [r,b]
            else:
                a = r    
                r = (a+b)/2
    #------------------------------------------------
    return r

## Método de punto fijo

Un punto fijo es un punto $p$ tal que $f(p) = p$.

Sea $p$ un punto fijo de la función $g(x)$ y sea $f(x)$ una función de la forma:

$$f(x)=x-g(x)$$

El punto $p$ es una raíz de la función $f(x)$. Para encontrar un punto fijo usamos la secuencia $p_n = g(p_n-1)$

In [7]:
def puntofijo(g,p0,eps = 1e-10,Nmax=100):
    '''Encuentra un punto fijo de la función g(x)'''
    
    i = 0
    while abs(g(p0)) >= eps and i < Nmax:
        i += 1
        p0 = g(p0) #encontramos un punto fijo usando la secuencia
    
    return p0

No todas las funciones convergen a un punto fijo.

## Teorema del punto fijo

Sea $g(x)$ una función continua en $(a,b)$, si $g(x)$ cumple que:

* $a < g(p_0) < b$ Para todo $p_0 \in (a,b)$
* $|g'(p_0)| < 1$

Entonces la secuencia $p_n = g(p_{n-1})$ converge a un único punto llamado **punto fijo**.

**nota:** Despejar $x$ de mayor grado

## Newton-Raphson

Consideremos el polinomio de Taylor hasta grado 2.

$$f(p) = f(p_0)+(p-p_0)f'(p_0)+\frac{(p-p_0)^2}{2}f''(\xi(p))$$

Como $p$ es una raiz de $f(x)$, $f(p) = 0$ y despejando $p$ tenemos:

$$ p = p_0 - \frac{f(p_0)}{f'(p_0)}$$

Para que este método funcione se debe garantizar que $f'(p_0) \neq 0 $

Para no complicarnos la vida con la derivada, podemos aproximarla usando el cociente incremental de Newton:

$$f(x)\approx \frac{f(x+h)-f(x)}{h}$$

In [26]:
def Newton(f,p0,h = 1e-5,eps = 1e-8,Nmax=100):
    '''Encuentra la raiz de la función usando el método de Newton-Raphson'''
    
    #encontremos la aproximación de la derivada
    df = (f(p0+h)-f(p0))/h
    
    #Debemos verificar que la derivada sea distinta de cero en p0
    if abs(df) <= eps:
        print('Elige otro p0!')
        p0 = None
    
    #implementamos el algoritmo
    else:
        i = 0
        while abs(f(p0)) >= eps and i <= Nmax:
            p0 = p0 - f(p0)/df
            df = (f(p0+h)-f(p0))/h
            i += 1
    return p0

## Método de la secante 

El método de la secante reemplaza la aproximación de la derivada por la siguiente expresión:

$$f'(p_{n-1}) \approx \frac{f(p_{n-1})-f(p_{n-2})}{p_{n-1}-p_{n-2}}$$

Así, el $h$ va variando en cada iteración y la expresión queda:

$$p_n = p_{n-1}-\frac{f(p_{n-1})(p_{n-1}-p_{n-2})}{f(p_{n-1})-f(p_{n-2})}$$

In [38]:
def Secante(f,p0,p1, eps = 1e-8,Nmax=100):
    '''Encuentra la raiz de la función usando el método de Newton-Raphson'''
    
    #Debemos verificar que la derivada sea distinta de cero en p0
    if abs(p1-p0) <= eps:
        p1 = None
    
    #implementamos el algoritmo
    else:
        i = 0
        while abs(f(p1)) >= eps and i <= Nmax:
            p2 = p0 - f(p0)*(p1-p0)/(f(p1)-f(p0))
            p0 = p1
            p1 = p2
    
            i += 1
    return p2

## Método de la falsa posición

Este método genera aproximaciones como el método de la secante pero incluye un test para asegurar que la raiź está siempre entre los corchetes de las sucesivas interacciones así como el método de bisección.

Este método es más lento que el de Newton y el de la secante, pero es más rápido que el de bisección y **garantiza convergencia** cuando existe la raiz.

el método funciona de la siguiente forma:

Dados dos puntos $p_0$ y $p_1$

1.) Si $f(p_0)f(p_1)< 0$, existe una raiz entre los puntos dados. Elegimos $p_2$ como el intercepto de la línea que une los puntos $(p_0,f(p_0))$ y $(p_1,f(p_1))$ con el eje $x$.

2.) Si $f(p_2)f(p_1) < 0$ entonces la raiz está entre $p_1$ y $p_2$, se elige $p_3$ como el intercepto en el eje x de la línea que une los puntos $p_1$ y $p_2$

* Si no, entonces $p_3$ se elige como el intercepto con el eje x de la línea que une los puntos $p_0$ y $p_2$

El procedimiento se repite hasta cumplir con la tolerancia

In [41]:
def  falsa_pos(f,p0,p1,eps=1e-8):
  '''Retorna la raiz de una función usando el método de la falsa posición'''
  
  #Verificamos que el intervalo sea válido

  #Si el intervalo no es válido imprimimos mensaje de error
  if p1 < p0 or f(p0)*f(p1)>0: 
    print('El intervalo no es válido')
  
  #Si el intervalo es válido, implementamos el algoritmo
  else:

    #definimos contador de iteraciones
    i = 0
    
    #mientras no se cumpla el maximo de iteraciones
    while i < 50:
      
      #aumentamos el contador en 1
      i += 1      
      
      #definimos los puntos iniciales
      f0 = f(p0)
      f1 = f(p1)
      
      #definimos p2 (intercepto eje x con recta que pasa por los puntos iniciales)
      p2 = p1 - f1*(p1-p0)/(f1-f0)
      f2 = f(p2)
      #Si el valor p2 cumple la tolerancia
      if abs(f(p2)) < eps:
        
        #rompemos el ciclo 
        break      
      
      #Si la raiz está en el intervalo [p2,p1]
      if f1*f2 < 0: 
          #redefinimos p0 como p1
          p0 = p1

      #redefinimos p1 como p2
      p1 = p2

    return p2,i

## Método de Steffensen

Variante del método de punto fijo el cual es acelerado mediante el uso del método de aitken.

### Aceleración de series de convergencia lineal (Aitken)

El método Aitken dice que si una secuencia $\{p_n\}_0^\infty$ es de convergencia lineal, esta se puede remplazar por la secuencia $\{p_n'\}_0^\infty$ generada por,
 
$$\boxed{p_n' = p_n − \frac{(p_{n+1} - p_n)^2}{p_{n+2} - 2p_{n+1} + p_n}}$$

la cual converge más rápido que la secuencia original.

El método nos dice que:

$$p\rightarrow p_n' \approx  p_n − \frac{(p_{n+1} - p_n)^2}{p_{n+2} - 2p_{n+1} + p_n}.$$

In [49]:
def Steffensen(f,p0, eps = 1e-8,Nmax=100):
    '''Encuentra la raiz de la función usando el método de Newton-Raphson'''
    
    #Debemos verificar que la derivada sea distinta de cero en p0
    if abs(f(p0)) <= eps:
        p1 = None
    
    #implementamos el algoritmo
    else:
        i = 0
        p1 = f(p0)
        p2 = f(p1)
        while abs(f(p2)) >= eps and i <= Nmax:
            p2 = p0 - (p1-p2)**2/(p2-2*p1+p0)
            p0 = p2
            p1 = f(p0)
            p2 = f(p1)
    
            i += 1
    return p2